# Lab 03: Prompt Caching

## Overview

In this notebook, we enable **prompt caching** to reduce costs on repeated requests. Prompt caching stores the processed system prompt and tool definitions, so subsequent requests can reuse them at a 90% discount.

**What you'll learn:**
- How to configure system prompt caching
- How to enable tool definition caching
- How to verify cache hits in Langfuse
- How to calculate cost savings from caching

**Optimizations in this notebook:**
- `SystemContentBlock` with `cachePoint`
- `cache_tools="default"` on BedrockModel

## Prerequisites

- Completed Labs 01-02

## Workshop Journey

```
01 Baseline → 02 Quick Wins → [03 Caching] → 04 Routing → 05 Guardrails → 06 Gateway → 07 Evaluations
                                   ↑
                              You are here
```

## Step 1: Setup

In [ ]:
import os
import json
import uuid
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_HOST', 'Not set')}")

## Step 2: Understanding Prompt Caching

### How Prompt Caching Works

1. **First request**: System prompt and tools are processed and cached
   - You pay full price + 25% cache write fee
   - Tokens appear as `cacheWriteInputTokens`

2. **Subsequent requests**: Cached content is reused
   - You pay only 10% of the normal input token price
   - Tokens appear as `cacheReadInputTokens`

### Cost Savings Example

For 1000 tokens of system prompt + tools:
- **Without caching**: 1000 tokens × $3.00/1M = $0.003 per request
- **With caching (first request)**: 1000 tokens × $3.75/1M = $0.00375
- **With caching (subsequent)**: 1000 tokens × $0.30/1M = $0.0003

After just 2 requests, caching pays for itself!

In [ ]:
# Review the caching configuration in v3 agent
agent_file = Path("agents/v3_caching.py")
print(agent_file.read_text())

## Step 3: Deploy the Caching Agent

In [ ]:
agent_name = "customer_support_v3_caching"
agent_file = str(Path("agents/v3_caching.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Agent name: {agent_name}")
print(f"Agent file: {agent_file}")
print(f"Requirements: {requirements_file}")

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

In [ ]:
# Modify Dockerfile for Langfuse
dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    # Replace opentelemetry-instrument wrapper with direct python call
    # Keep the correct module path using regex
    if "opentelemetry-instrument" in content:
        import re
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]',
            r'CMD ["python", "-m", "\1"]',
            content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")
else:
    print("Dockerfile not found - will be created during deployment")

In [ ]:
env_vars = {
    "LANGFUSE_HOST": os.environ.get("LANGFUSE_HOST"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

## Step 4: Test Caching Behavior

We'll run the same query multiple times to demonstrate cache hits.

In [ ]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [ ]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    get_latest_trace_metrics,
    print_metrics,
    clear_metrics,
    collect_metric,
    print_metrics_table,
    get_collected_metrics
)

# Clear any previously collected metrics
clear_metrics()

# Standard test prompts - same across all notebooks for consistent comparison
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),

    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),

    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),

    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),

    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
# First request should show cache WRITE, subsequent should show cache READ
for i, (test_name, prompt) in enumerate(TEST_PROMPTS):
    print("=" * 60)
    print(f"Test {i+1}: {test_name}")
    if i == 0:
        print("(First request - Cache WRITE expected)")
    else:
        print("(Subsequent request - Cache READ expected)")
    print("=" * 60)

    response = invoke_agent(prompt)
    print(response)

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v3-caching",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

In [ ]:
# Print summary table
print_metrics_table()

# Compare with baseline metrics (from notebook 01)
BASELINE_AVG_INPUT_TOKENS = 4251  # From v1-baseline
BASELINE_AVG_LATENCY = 8.0  # From v1-baseline (seconds)

# Calculate improvements
collected = get_collected_metrics()
if collected:
    valid_metrics = [m for m in collected if "error" not in m]
    if valid_metrics:
        avg_input = sum(m.get('input_tokens', 0) for m in valid_metrics) / len(valid_metrics)
        avg_latency = sum(m.get('latency_seconds', 0) or 0 for m in valid_metrics) / len(valid_metrics)
        total_cost = sum(m.get('cost_usd', 0) for m in valid_metrics)
        total_cache_read = sum(m.get('cache_read_tokens', 0) for m in valid_metrics)
        total_cache_write = sum(m.get('cache_write_tokens', 0) for m in valid_metrics)

        token_reduction = ((BASELINE_AVG_INPUT_TOKENS - avg_input) / BASELINE_AVG_INPUT_TOKENS) * 100
        latency_change = ((BASELINE_AVG_LATENCY - avg_latency) / BASELINE_AVG_LATENCY) * 100

        print("\n" + "=" * 60)
        print("           COMPARISON VS BASELINE (v1)")
        print("=" * 60)
        print(f"  Avg Input Tokens:    {avg_input:,.0f} (Baseline: {BASELINE_AVG_INPUT_TOKENS:,})")
        print(f"  Token Reduction:     {token_reduction:+.1f}%")
        print(f"  Avg Latency:         {avg_latency:.2f}s (Baseline: {BASELINE_AVG_LATENCY:.2f}s)")
        print(f"  Latency Change:      {latency_change:+.1f}%")
        print(f"  Total Cost:          ${total_cost:.4f}")
        print("-" * 60)
        print(f"  Cache Write Tokens:  {total_cache_write:,} (first request)")
        print(f"  Cache Read Tokens:   {total_cache_read:,} (subsequent requests)")
        print("=" * 60)

In [ ]:
# Run 3: Third request - should also show cache READ
print("=" * 60)
print("Run 3: Third Request (Cache Read Expected)")
print("=" * 60)
response = invoke_agent("Tell me about your tablets")
print(response)
print("\nCheck Langfuse: cacheReadInputTokens should be > 0")

## Step 5: Analyze Cache Metrics in Langfuse

Open Langfuse and examine the token metrics for each request.

## Step 6: Calculate Cost Savings

In [ ]:
# Cost calculation helper
def calculate_caching_savings(cached_tokens, num_requests):
    """Calculate savings from prompt caching."""
    # Sonnet pricing per 1M tokens
    input_price = 3.00
    cache_write_price = 3.75  # 25% premium
    cache_read_price = 0.30   # 90% discount
    
    # Without caching
    cost_no_cache = (cached_tokens / 1_000_000) * input_price * num_requests
    
    # With caching
    cost_first_request = (cached_tokens / 1_000_000) * cache_write_price
    cost_subsequent = (cached_tokens / 1_000_000) * cache_read_price * (num_requests - 1)
    cost_with_cache = cost_first_request + cost_subsequent
    
    savings = cost_no_cache - cost_with_cache
    savings_pct = (savings / cost_no_cache) * 100
    
    return {
        "without_caching": cost_no_cache,
        "with_caching": cost_with_cache,
        "savings": savings,
        "savings_pct": savings_pct,
    }

# Example: 500 cached tokens, 10 requests
result = calculate_caching_savings(cached_tokens=500, num_requests=10)
print("Cost Comparison (500 cached tokens, 10 requests):")
print(f"  Without caching: ${result['without_caching']:.6f}")
print(f"  With caching:    ${result['with_caching']:.6f}")
print(f"  Savings:         ${result['savings']:.6f} ({result['savings_pct']:.1f}%)")

In [ ]:
# At scale: 1000 requests per day
result = calculate_caching_savings(cached_tokens=500, num_requests=1000)
print("Cost Comparison (500 cached tokens, 1000 requests/day):")
print(f"  Without caching: ${result['without_caching']:.4f}/day")
print(f"  With caching:    ${result['with_caching']:.4f}/day")
print(f"  Daily savings:   ${result['savings']:.4f} ({result['savings_pct']:.1f}%)")
print(f"  Monthly savings: ${result['savings'] * 30:.2f}")

## Summary

In this notebook, we enabled prompt caching:

1. **System prompt caching**: Using `SystemContentBlock` with `cachePoint`
2. **Tool caching**: Using `cache_tools="default"`

**Key Observations:**
- First request: Cache write (25% premium)
- Subsequent requests: Cache read (90% discount)
- Break-even after just 2 requests

**Next Steps:** In the next notebook, we'll add model routing to use cheaper models for simple queries.

**Next notebook:** [04-llm-routing.ipynb](./04-llm-routing.ipynb)